In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pylab as plt
import matplotlib.ticker as ticker
from datetime import datetime
import matplotlib.dates as mdates
from matplotlib.lines import Line2D   #for custom legend
import math
import statsmodels.api as am
from statsmodels.tsa.stattools import acf, pacf
import statsmodels.tsa.stattools as ts
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as an 
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from statsmodels.tsa.arima.model import ARIMA
from pandas.plotting import autocorrelation_plot
from sklearn.metrics import mean_squared_error
import warnings
warnings.filterwarnings("ignore")
from matplotlib import pyplot
from datetime import datetime, timedelta

# Function to add dark periods to the plot
def add_dark_periods(ax, times, period_start='18:00', period_end='06:00'):
    """
    Adds shaded areas to the plot to represent dark periods.
    
    Parameters:
    - ax: The axis to add the shaded areas to.
    - times: Series of datetime objects.
    - period_start: Start time of the dark period each day (HH:MM format).
    - period_end: End time of the dark period each day (HH:MM format).
    """
    for time in times:
        day_start = datetime.combine(time.date(), datetime.min.time())
        dark_start = datetime.combine(time.date(), datetime.strptime(period_start, '%H:%M').time())
        dark_end = datetime.combine(time.date(), datetime.strptime(period_end, '%H:%M').time())
        
        if dark_start > dark_end:  # Handle period spanning midnight
            dark_start_next = dark_start
            dark_end_next = dark_end + timedelta(days=1)
            ax.axvspan(dark_start_next, dark_end_next, color='grey', alpha=0.3)
        else:
            ax.axvspan(dark_start, dark_end, color='grey', alpha=0.3)


# Read the CSV file
def parser(x):
    return datetime.strptime('%Y/%m/%d %H:%M:%S')
dreadd_df = pd.read_csv('/Volumes/NO_NAME/Personal/Aaron_Cone_Data_and_Code/Locomotion.csv', date_parser=parser)

# Converts DateTime and Time Columns to a datetime format
dreadd_df['Time'] = pd.to_datetime(dreadd_df['Time'])



# Round to nearest 10th minute
dreadd_df['Time'] = dreadd_df['Time'].round('10min')


# Makes sure figure is correctly sized
fig, ax = plt.subplots(figsize = (20,5))


# Plots original data to take a look at it
plt.plot(dreadd_df['Time'],dreadd_df['Loco'])
plt.title('Original Data')
plt.ylabel('Locomotion (counts/hr)')
plt.xlabel('Time (days)')

# Handles DateTime
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=4))

# Rotate x tick labels
plt.xticks(rotation=45)
                             
# Adding dark periods based on 'Time' column
add_dark_periods(ax, dreadd_df['Time'])
    

# Plots autocorrelation and partial correlation 
plot_acf(dreadd_df['Loco'])
plt.xlabel('Lag')
plot_pacf(dreadd_df['Loco'])
plt.xlabel('Lag')

# Plots and fits model
ARIMAmodel_dreadd_df_rer_fit = ARIMA(dreadd_df['Loco'], order = (5,2,1)).fit()

# Summarizes model
print(ARIMAmodel_dreadd_df_rer_fit.summary())


# Plots residual values of the model
residuals = pd.DataFrame(ARIMAmodel_dreadd_df_rer_fit.resid)
residuals.plot()

# Plot density of residual values and summary of residual values
residuals.plot(kind='kde')
print(residuals.describe())

# Splits data into trains and test trains 
X = dreadd_df['Loco'].values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# Forecasts model
for t in range(len(test)):
    model = ARIMA(history, order=(3,1,0))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print(f'predicted={yhat}, expected={obs}')

# Evaluates forecasted model
rmse = math.sqrt(mean_squared_error(test, predictions))
print(f'Test RMSE: {rmse:.2f}%')

# Plots forecasted model vs the actual values
pyplot.plot(test)
pyplot.plot(predictions, color='red')
pyplot.show()

# Evaluate ARIMA model for a given order (p,d,q)
def evaluate_arima_model(X, arima_order):
    "Prepares training dataset"
    train_size = int(len(X) * 0.66)
    train, test = X[0:train_size], X[train_size:]
    history = [x for x in train]
    
    # Makes predictions from model
    predictions = list()
    for t in range(len(test)):
        model = ARIMA(history, order=arima_order)
        model_fit = model.fit()
        yhat = model_fit.forecast()[0]
        predictions.append(yhat)
        history.append(test[t])
        
    # Calculates out-of-sample error
    rmse = math.sqrt(mean_squared_error(test, predictions))
    return rmse
 
    # Evaluates combinations of p, d and q values for an ARIMA model
def evaluate_models(dataset, p_values, d_values, q_values):
    dataset = dataset.astype('float32')
    best_score, best_cfg = float("inf"), None
    for p in p_values:
        for d in d_values:
            for q in q_values:
                order = (p, d, q)
                try:
                    rmse = evaluate_arima_model(dataset, order)
                    if rmse < best_score:
                        best_score, best_cfg = rmse, order
                    print(f'ARIMA{order} RMSE={rmse:.3f}')
                except:
                    continue
    print(f'Best ARIMA{best_cfg} RMSE={rmse:.3f}')

# Evaluates parameters mentioned above
p_values = [0, 1, 2, 4]
d_values = range(0, 3)
q_values = range(0, 3)
evaluate_models(dreadd_df['Loco'].values, p_values, d_values, q_values)


# Grid Searching Method

# Iterates over the model

# Split into train and test sets
X = dreadd_df['Loco'].values
size = int(len(X) * 0.66)
train, test = X[0:size], X[size:len(X)]
history = [x for x in train]
predictions = list()

# Walk-forward validation 
for t in range(len(test)):
    model = ARIMA(history, order=(1, 2, 2))
    model_fit = model.fit()
    output = model_fit.forecast()
    yhat = output[0]
    predictions.append(yhat)
    obs = test[t]
    history.append(obs)
    print(f'predicted={yhat:.1f}, expected={obs}')



In [ ]:
# Makes sure figure is correctly sized
fig, ax = plt.subplots(figsize = (20,5))

# Subset data
subset_df = dreadd_df.head(244)

# Plots original data to take a look at it
plt.plot(subset_df['Time'], test)
plt.plot(subset_df['Time'],predictions, color='red')
plt.title('1.5 Days Future Prediction of Locomotion')
plt.ylabel('Locomotion (counts/hr)')
plt.xlabel('Time (days)')

# Handles DateTime
ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M'))
ax.xaxis.set_major_locator(mdates.HourLocator(interval=4))

# Rotate x tick labels
plt.xticks(rotation=45)
                             
# Adding dark periods based on 'Time' column
add_dark_periods(ax, subset_df['Time'])

plt.show()